In [ ]:
import os
tuh_sz_path = "/mnt/share/data/tuh_raw/tuh_eeg_seizure/v2.0.0/edf/"
plot_args_path = "./tusz-plotinfo/"; os.makedirs(plot_args_path, exist_ok=True)
dt_fmt = '%Y-%m-%d %H:%M:%S'
ignore_lst = []

import os
import json, csv
import glob
from datetime import datetime, timedelta
import pyedflib
import pandas as pd

with os.scandir(tuh_sz_path) as entries:
    for entry in entries:
        if entry.is_dir():
            # subdir_path = entry.path
            with os.scandir(entry.path) as pats:
                for pat in pats:
                    if pat.is_dir():
                        result_obj = {"record_lst": [], "seizure_lst": [], "unused_rec_idx_lst": []}
                        record_fn_lst =  glob.glob(os.path.join(pat.path, "**", "*.edf"), recursive=True)
                        
                        for edf_path in record_fn_lst:
                            pedf = pyedflib.EdfReader(edf_path)
                            start_dt = pedf.getStartdatetime()
                            end_dt = start_dt + timedelta(seconds=(edf_len := pedf.getFileDuration()))
                            fs = pedf.getSampleFrequency(0)
                            # assert all((FS := pedf.getSampleFrequencies()) == fs) # TODO 支持过滤非脑电数据通道
                            result_obj["record_lst"].append({
                                "file": os.path.basename(edf_path), 
                                # "span": [start_dt.strftime(dt_fmt), end_dt.strftime(dt_fmt)], 
                                "span": [start_dt, end_dt], 
                                "info": f"{os.path.basename(edf_path)} of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
                            })
                            pedf.close()
                            with open(edf_path[:-3]+"csv_bi", "rt") as fcsv:
                                rows = (row for row in fcsv if not row.startswith('#') and row.strip() != '')
                                reader = csv.DictReader(rows)
                                for row in reader:
                                    if row["label"] == "bckg":
                                        continue
                                    # assert row["label"] == "seiz"
                                    # assert row["channel"] == "TERM"
                                    # assert 0 <= float(row["start_time"]) < float(row["stop_time"]) <= edf_len
                                    result_obj["seizure_lst"].append({
                                        "span": [start_dt+timedelta(seconds=float(row["start_time"])), start_dt+timedelta(seconds=float(row["stop_time"]))], 
                                        "info": f'Onset {start_dt+timedelta(seconds=float(row["start_time"]))}, last {float(row["stop_time"]) - float(row["start_time"])}s'
                                    })
                            # TODO Add more verification
                        result_obj["record_lst"].sort(key=lambda obj:obj["span"])
                        result_obj["seizure_lst"].sort(key=lambda obj:obj["span"])
                        for k, rec_info in enumerate(result_obj["record_lst"]):
                            if rec_info["file"] in ignore_lst:
                                result_obj["unused_rec_idx_lst"].append(k)
                            rec_info["span"] = [rec_info["span"][0].strftime(dt_fmt), rec_info["span"][1].strftime(dt_fmt)]
                        
                        for seiz_info in result_obj["seizure_lst"]:
                            seiz_info["span"] = [seiz_info["span"][0].strftime(dt_fmt), seiz_info["span"][1].strftime(dt_fmt)]
                            
                        with open(os.path.join(plot_args_path, f'{pat.name}_sz{len(result_obj["seizure_lst"])}_edf{len(result_obj["record_lst"])}.json'), "wt") as fout:
                            json.dump(result_obj, fout, indent=2)




In [1]:
# TODO Use plotly.py Interactive Widgets 
# extra_gap_consideration = True
# range_type = boundry_calc_method.asym_relative
import plotly.graph_objects as go
# import pandas as pd
# import numpy as np
from datetime import timedelta

from typing import Sequence, Optional, Set, Tuple

SegInfo = None
def get_pat_timeline(title: str, record_seq: SegInfo, seizure_seq: SegInfo, unused_set: Set[str] = {}, **kwargs) -> go.Figure:
    onset_points = go.Scatter(x=tuple(ost["span"][0] for ost in seizure_seq), y=[0] * len(seizure_seq), mode="markers", hovertext=tuple(ost["info"] for ost in seizure_seq), 
                    xaxis='x', yaxis='y', name="Onset", legendrank=1e1, 
                    marker_size=8, marker_color="orange")

    ### Generate Rects Attr
    base = [seg["span"][0] for seg in record_seq]
    length = [(seg["span"][1] - seg["span"][0]).total_seconds() * 1000 for seg in record_seq] # Unit(ms)
    text = [seg["info"] for seg in record_seq]
    color = ["MediumSeaGreen" if seg["file"] not in unused_set else "DarkMagenta" for seg in record_seq]
    # opacity = [1.0] * len(record_seq)
    shape = []; 
    for seg in record_seq:
        if seg["file"] in unused_set:    shape.append('x')
        elif "masked_set" in kwargs and seg["file"] in kwargs["masked_set"]:  shape.append('.')
        else:                       shape.append('')


    offset=-4; width=4# ; ymin=offset+width
    recs_bar = go.Bar(
        base=base,
        x=length, 
        y=[0] * len(record_seq),
        hovertext=text, 
        orientation='h', 
        marker=dict(
            color=color,
            # opacity=opacity, 
            pattern_shape=shape, 
        ), 
        name="Record", 
        offset=offset, 
        width=[width] * len(record_seq), 
    )

    base = [seg["span"][0] for seg in seizure_seq]
    length = [(seg["span"][1] - seg["span"][0]).total_seconds() * 1000 for seg in seizure_seq]
    text = [seg["info"] for seg in seizure_seq]
    color = "Gold"

    offset=0; width=4# ; ymin=offset+width
    szs_bar = go.Bar(
        base=base,
        x=length, 
        y=[0] * len(seizure_seq),
        hovertext=text, 
        orientation='h', 
        marker=dict(
            color=color,
            # opacity=opacity, 
        ), 
        name="Seizure", 
        offset=offset, 
        width=[width] * len(seizure_seq), 
    )    

    fig = go.Figure(data=[onset_points, recs_bar, szs_bar], layout_title=title)
    if len(seizure_seq) == 0: fig.update_xaxes(type='date')

    ### Emphasize xaxis & Onset Point
    fig.add_hline(y=0, line_color="gray", line_width=1)

    ### Ajust global Layout
    fig.update_layout(
        yaxis=dict(
            range=[-8, 8] if "yaxis_range" not in kwargs else kwargs["yaxis_range"], 
            fixedrange=False, 
        )
    )
    fig.update_xaxes(
        range=[record_seq[0]["span"][0] - timedelta(hours=1), record_seq[-1]["span"][1] + timedelta(hours=1)], 
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=6, label="6h", step="hour", stepmode="backward"),
                dict(count=1, label="对齐到天", step="day",  stepmode="todate"),
                dict(count=1, label="1d", step="day",  stepmode="backward"),
            ])
        )
    )

    return fig

In [2]:
import re # Add TUH-EEG Extra Record Info
import os
import glob
import json
from datetime import datetime, timedelta
import plotly.offline as pyo
import plotly.graph_objs as go
import pyedflib

dt_fmt = '%Y-%m-%d %H:%M:%S'
tuh_path = "/mnt/share/data/tuh_raw/tuh_eeg/v2.0.0/edf/"
plot_args_path = "./tusz-plotinfo/"
plotly_fig_path = "./tusz-timeline/"

def extract_values_from_pattern(pattern):
    regex = r'(\w+)_sz(\d+)_edf(\d+).json'
    match = re.match(regex, pattern)
    
    if match:
        pat = match.group(1)
        sz_cnt = int(match.group(2))
        edf_cnt = int(match.group(3))
        
        return sz_cnt, edf_cnt, pat
    
json_lst = list(os.scandir(plot_args_path))
json_lst.sort(key=lambda fe: extract_values_from_pattern(fe.name)[:2], reverse=True)

pat_2_complete_path = {extract_values_from_pattern(fe.name)[-1]: "" for fe in json_lst}
for first_level_entry in os.scandir(tuh_path):
    if first_level_entry.is_dir():
        for second_level_entry in os.scandir(first_level_entry):
            if second_level_entry.is_dir() and second_level_entry.name in pat_2_complete_path:
                pat_2_complete_path[second_level_entry.name] = second_level_entry.path

for fe in json_lst:
    with open(fe.path, "rt") as f:
        info_obj = json.load(f)
    pat_record_lst = info_obj["record_lst"]
    pat_seizure_lst = info_obj["seizure_lst"]
    for rec in pat_record_lst:
        rec["span"] = [datetime.strptime(rec["span"][0], dt_fmt), datetime.strptime(rec["span"][1], dt_fmt)]
    for rec in pat_seizure_lst:
        rec["span"] = [datetime.strptime(rec["span"][0], dt_fmt), datetime.strptime(rec["span"][1], dt_fmt)]
    fig = get_pat_timeline(fe.name, pat_record_lst, pat_seizure_lst)

    record_fn_lst =  glob.glob(os.path.join(pat_2_complete_path[extract_values_from_pattern(fe.name)[-1]], "**", "*.edf"), recursive=True)
    pat_complete_lst = []

    for edf_path in record_fn_lst:
        pedf = pyedflib.EdfReader(edf_path)
        start_dt = pedf.getStartdatetime()
        end_dt = start_dt + timedelta(seconds=(edf_len := pedf.getFileDuration()))
        fs = pedf.getSampleFrequency(0)
        # assert all((FS := pedf.getSampleFrequencies()) == fs) # TODO 支持过滤非脑电数据通道
        pat_complete_lst.append({
            "file": os.path.basename(edf_path), 
            # "span": [start_dt.strftime(dt_fmt), end_dt.strftime(dt_fmt)], 
            "span": [start_dt, end_dt], 
            "info": f"{os.path.basename(edf_path)} of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
        })
        pedf.close()

    base = [seg["span"][0] for seg in pat_complete_lst]
    length = [(seg["span"][1] - seg["span"][0]).total_seconds() * 1000 for seg in pat_complete_lst]
    text = [seg["info"] for seg in pat_complete_lst]
    color = "LightSeaGreen"

    offset=-8; width=4# ; ymin=offset+width
    fig.add_bar(
        base=base,
        x=length, 
        y=[0] * len(pat_complete_lst),
        hovertext=text, 
        orientation='h', 
        marker=dict(
            color=color,
            # opacity=opacity, 
        ), 
        name="Origin", 
        offset=offset, 
        width=[width] * len(pat_complete_lst), 
    )     
        
    # print(f"Prepare to plot {fe.name}")
    if len(pat_record_lst) != len(pat_complete_lst): print(f"{fe.name[:-4]} non equal!")
    pyo.plot(fig, filename=os.path.join(plotly_fig_path, fe.name[5:-4]+".html"), include_plotlyjs="./plotly.min.js", 
                auto_open=False, image='svg', image_width=2560, image_height=1440)

aaaaahie_sz202_edf234. non equal!
aaaaapks_sz202_edf94. non equal!
aaaaaqtx_sz139_edf38. non equal!
aaaaajqo_sz99_edf57. non equal!
aaaaajru_sz84_edf49. non equal!
aaaaahyp_sz82_edf15. non equal!
aaaaajdn_sz71_edf9. non equal!
aaaaates_sz67_edf51. non equal!
aaaaardk_sz66_edf63. non equal!
aaaaabhz_sz60_edf37. non equal!
aaaaardf_sz53_edf45. non equal!
aaaaakfo_sz52_edf21. non equal!
aaaaaksg_sz51_edf7. non equal!
aaaaafcf_sz50_edf10. non equal!
aaaaakkm_sz46_edf32. non equal!
aaaaasdq_sz42_edf31. non equal!
aaaaaqkb_sz41_edf41. non equal!
aaaaamtk_sz41_edf1. non equal!
aaaaanme_sz40_edf97. non equal!
aaaaarsm_sz38_edf47. non equal!
aaaaagpk_sz38_edf25. non equal!
aaaaaqtq_sz36_edf15. non equal!
aaaaatjh_sz30_edf16. non equal!
aaaaaghb_sz27_edf19. non equal!
aaaaakoe_sz27_edf11. non equal!
aaaaatez_sz26_edf29. non equal!
aaaaaarq_sz23_edf43. non equal!
aaaaafjw_sz23_edf22. non equal!
aaaaaooo_sz23_edf11. non equal!
aaaaajsl_sz23_edf10. non equal!
aaaaarpv_sz22_edf32. non equal!
aaaaaiz

In [1]:
from pyedflib import EdfReader

class EdfReaderWrapper(EdfReader):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __enter__(self):
        return self
    def __exit__(self, *args):
        super().close()

In [37]:
import mne

class MNEEdfObjWrapper:
    def __init__(self, *args, **kwargs):
        self.raw = mne.io.read_raw_edf(*args, **kwargs)
    def __enter__(self):
        return self.raw
    def __exit__(self, *args):
        self.raw.close()


In [47]:
import os
xuanwu_data_path = "./disk/"
plot_args_path = "./xuanwu-mneinfo/"; os.makedirs(plot_args_path, exist_ok=True)
dt_fmt = '%Y-%m-%d %H:%M:%S'
ignore_lst = []

import os
import json, csv
import glob
from datetime import datetime, timedelta
import pyedflib
import pandas as pd
import warnings

with os.scandir(xuanwu_data_path) as entries:
    for entry in entries:
                        result_obj = {"record_lst": [], "seizure_lst": [], "unused_rec_idx_lst": []}
                        record_fn_lst =  glob.glob(os.path.join(entry.path, "**", "*.edf"), recursive=True)
                        record_fn_lst.extend(glob.glob(os.path.join(entry.path, "**", "*.EDF"), recursive=True))
                        record_fn_lst.extend(glob.glob(os.path.join(entry.path, "**", "*.BDF"), recursive=True))
                        record_fn_lst.extend(glob.glob(os.path.join(entry.path, "**", "*.bdf"), recursive=True))
                        
                        for edf_path in record_fn_lst:
                            try:
                                print(f"Try loading {edf_path}")
                                # with EdfReaderWrapper(edf_path) as pedf:
                                #     start_dt = pedf.getStartdatetime()
                                #     end_dt = start_dt + timedelta(seconds=(edf_len := pedf.getFileDuration()))
                                #     fs = pedf.getSampleFrequency(0)
                                #     # assert all((FS := pedf.getSampleFrequencies()) == fs) # TODO 支持过滤非脑电数据通道
                                #     result_obj["record_lst"].append({
                                #         "file": os.path.basename(edf_path), 
                                #         # "span": [start_dt.strftime(dt_fmt), end_dt.strftime(dt_fmt)], 
                                #         "span": [start_dt, end_dt], 
                                #         "info": f"{os.path.basename(edf_path)} of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
                                #     })
                                with MNEEdfObjWrapper(edf_path, preload=False) as raw:
                                    start_dt = raw.info['meas_date']
                                    end_dt = start_dt + timedelta(seconds=(raw.n_times / raw.info['sfreq'])) # TODO 核查对于EDF-D情形下此算法是否正确
                                    fs = raw.info['sfreq']

                                    # 构建病人信息字典，并添加到列表中
                                    result_obj["record_lst"].append({
                                        "file": os.path.basename(edf_path),
                                        "span": [start_dt, end_dt],
                                        "info": f"{os.path.basename(edf_path)} of shape {len(raw.ch_names), raw.n_times}", 
                                        "annotations": [a['description'] for a in raw.annotations] if hasattr(raw, 'annotations') else []
                                    })                                    
                            except ValueError as exp:
                                warnings.warn(f"ValueError from {exp}")
                            # with open(edf_path[:-3]+"csv_bi", "rt") as fcsv:
                            #     rows = (row for row in fcsv if not row.startswith('#') and row.strip() != '')
                            #     reader = csv.DictReader(rows)
                            #     for row in reader:
                            #         if row["label"] == "bckg":
                            #             continue
                            #         # assert row["label"] == "seiz"
                            #         # assert row["channel"] == "TERM"
                            #         # assert 0 <= float(row["start_time"]) < float(row["stop_time"]) <= edf_len
                            #         result_obj["seizure_lst"].append({
                            #             "span": [start_dt+timedelta(seconds=float(row["start_time"])), start_dt+timedelta(seconds=float(row["stop_time"]))], 
                            #             "info": f'Onset {start_dt+timedelta(seconds=float(row["start_time"]))}, last {float(row["stop_time"]) - float(row["start_time"])}s'
                            #         })
                            # TODO Add more verification
                        result_obj["record_lst"].sort(key=lambda obj:obj["span"])
                        result_obj["seizure_lst"].sort(key=lambda obj:obj["span"])
                        for k, rec_info in enumerate(result_obj["record_lst"]):
                            if rec_info["file"] in ignore_lst:
                                result_obj["unused_rec_idx_lst"].append(k)
                            rec_info["span"] = [rec_info["span"][0].strftime(dt_fmt), rec_info["span"][1].strftime(dt_fmt)]
                        
                        for seiz_info in result_obj["seizure_lst"]:
                            seiz_info["span"] = [seiz_info["span"][0].strftime(dt_fmt), seiz_info["span"][1].strftime(dt_fmt)]
                            
                        with open(os.path.join(plot_args_path, f'{entry.name}.json'), "wt") as fout:
                            json.dump(result_obj, fout, indent=2)




Try loading ./disk/CuiYibing/INTERICTAL/WAKING/CuiYiBing_WAKINGNEW2.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibing/INTERICTAL/WAKING/CuiYiBing_WAKINGNEW2.edf...
EDF file detected
Try loading ./disk/CuiYibing/INTERICTAL/WAKING/CuiYiBing_WAKINGNEW1.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibing/INTERICTAL/WAKING/CuiYiBing_WAKINGNEW1.edf...
EDF file detected
Try loading ./disk/CuiYibing/INTERICTAL/SLEEPING/CYB_SLEEPING_add electrode.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibing/INTERICTAL/SLEEPING/CYB_SLEEPING_add electrode.edf...
EDF file detected


Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_252003/1187373617.py:50: UserWarning:

ValueError from invalid literal for int() with base 10: '\x00\x00\x00\x00\x00\x00\x00\x00'



Try loading ./disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0003.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0003.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0002.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0005.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0005.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/CuiYibing/INTERICTAL/SLEEPING/CYB_Sleeping_add electrode_0001.edf
Extracting EDF parameters from /home/bigeye/disk/CuiYibin

/tmp/ipykernel_252003/1187373617.py:50: UserWarning:

ValueError from invalid literal for int() with base 10: ''



...

Try loading ./disk/Cui_YiBing_原始/SEIZURE/SEIZURE5/Cui~ YiBing_ebff4518-6ac6-4407-a3a5-066c3fe3a39e.edf
Extracting EDF parameters from /home/bigeye/disk/Cui_YiBing_原始/SEIZURE/SEIZURE5/Cui~ YiBing_ebff4518-6ac6-4407-a3a5-066c3fe3a39e.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_252003/4239133375.py:5: RuntimeWarning:

Omitted 1 annotation(s) that were outside data range.



Try loading ./disk/Cui_YiBing_原始/SEIZURE/SEIZURE3/Cui~ YiBing_14417a59-7060-46fc-9c74-b9191d3d5599.edf
Extracting EDF parameters from /home/bigeye/disk/Cui_YiBing_原始/SEIZURE/SEIZURE3/Cui~ YiBing_14417a59-7060-46fc-9c74-b9191d3d5599.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/Cui_YiBing_原始/SEIZURE/SEIZURE1/Cui~ YiBing_c6aca80c-1988-4654-8f91-990525c306a4.edf
Extracting EDF parameters from /home/bigeye/disk/Cui_YiBing_原始/SEIZURE/SEIZURE1/Cui~ YiBing_c6aca80c-1988-4654-8f91-990525c306a4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/Cui_YiBing_原始/SEIZURE/SEIZURE6/Cui~ YiBing_cefbf70f-ed3e-4545-99da-a159462dd75f.edf
Extracting EDF parameters from /home/bigeye/disk/Cui_YiBing_原始/SEIZURE/SEIZURE6/Cui~ YiBing_cefbf70f-ed3e-4545-99da-a159462dd75f.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/Cui_YiBing_原始/S

/tmp/ipykernel_252003/4239133375.py:5: RuntimeWarning:

Omitted 2 annotation(s) that were outside data range.



Creating raw.info structure...
Try loading ./disk/ShiLanxin/SZ/SZ3/AFTER_2017_0221_7AM/lanxin~ shi_ed39387c-e97a-4a37-8ee1-5248ed12875b.edf
Extracting EDF parameters from /home/bigeye/disk/ShiLanxin/SZ/SZ3/AFTER_2017_0221_7AM/lanxin~ shi_ed39387c-e97a-4a37-8ee1-5248ed12875b.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/ShiLanxin/SZ/SZ2/PRESZ_2017_02_21_1am/lanxin~ shi_ca8120b7-40e9-4366-b9d1-fc43d55b35f9.edf
Extracting EDF parameters from /home/bigeye/disk/ShiLanxin/SZ/SZ2/PRESZ_2017_02_21_1am/lanxin~ shi_ca8120b7-40e9-4366-b9d1-fc43d55b35f9.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Try loading ./disk/ShiLanxin/SZ/SZ2/presz_2017_0221_0am/lanxin~ shi_578f519c-5db4-4ed4-a298-a0a842040a94.edf
Extracting EDF parameters from /home/bigeye/disk/ShiLanxin/SZ/SZ2/presz_2017_0221_0am/lanxin~ shi_578f519c-5db4-4ed4-a298-a0a842040a94.edf...
EDF file detected
Setting channel info structure